In [1]:
%matplotlib inline
import pickle
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt

from patsy import dmatrices
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score, cross_val_predict

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, classification_report

### Open previously saved pickle file

In [2]:
with open('un_explore.pkl') as picklefile:
    undata = pickle.load(picklefile)

### Print column names and counts for non-NaN values for each column

In [3]:
for col in sorted(undata.columns):
    print undata[col].count(), col

2814 adolescent_birth_rate_per_1000_women
302 agriculture_support_estimate_for_oecd_countries_as_percentage_of_their_gdp
302 agriculture_support_estimate_for_oecd_countries_million_us
2808 aids_deaths
2808 aids_deaths_lower_bound
2808 aids_deaths_upper_bound
936 aids_orphans_one_or_both_parents
349 antenatal_care_coverage_at_least_four_visits_percentage
645 antenatal_care_coverage_at_least_one_visit_percentage
310 antiretroviral_therapy_coverage_among_people_with_advanced_hiv_infection_percentage
316 antiretroviral_therapy_coverage_among_people_with_advanced_hiv_infection_percentage_lower_bound
322 antiretroviral_therapy_coverage_among_people_with_advanced_hiv_infection_percentage_upper_bound
157 condom_use_at_last_highrisk_sex_1524_years_old_men_percentage
191 condom_use_at_last_highrisk_sex_1524_years_old_women_percentage
144 condom_use_population_ages_1524_female__of_females_ages_1524
122 condom_use_population_ages_1524_male__of_males_ages_1524
682 condom_use_to_overall_contraceptiv

### Print counts specifically for contraceptive use variables

In [4]:
contr_use = ['condom_use_at_last_highrisk_sex_1524_years_old_men_percentage',
    'condom_use_at_last_highrisk_sex_1524_years_old_women_percentage',
    'condom_use_population_ages_1524_female__of_females_ages_1524',
    'condom_use_population_ages_1524_male__of_males_ages_1524',
    'condom_use_to_overall_contraceptive_use_among_currently_married_women_1549_years_old_percentage',
    'condom_use_with_non_regular_partner__adults1549_female',
    'condom_use_with_non_regular_partner__adults1549_male',
    'contraceptive_prevalence__of_women_ages_1549']

In [5]:
for col in contr_use:
    print undata[col].count(), col

157 condom_use_at_last_highrisk_sex_1524_years_old_men_percentage
191 condom_use_at_last_highrisk_sex_1524_years_old_women_percentage
144 condom_use_population_ages_1524_female__of_females_ages_1524
122 condom_use_population_ages_1524_male__of_males_ages_1524
682 condom_use_to_overall_contraceptive_use_among_currently_married_women_1549_years_old_percentage
125 condom_use_with_non_regular_partner__adults1549_female
120 condom_use_with_non_regular_partner__adults1549_male
786 contraceptive_prevalence__of_women_ages_1549


### Create funtion for linear regression modeling

The `model_lr_data` function creates matrices and a linear regression model and outputs a summary of the results. The inputs are:

* `dep`: dependent variable
* `ind`: a list of features / independent variables
* `data`: which dataframe to use

In [6]:
def model_lr_data(dep, ind, data):
    formula = dep + '~' + '+'.join(ind)
    y, X = dmatrices(formula, data=data, return_type='dataframe' )
    print X.shape
    print '\n****PREDICTING\n' + dep
    model = sm.OLS(y, X)
    results = model.fit()
    print results.summary()

In [7]:
hiv = 'hiv_incidence_rate_1549_years_old_percentage_midpoint'
pop = 'population_total'
oda = 'net_oda_mill'
gni = 'gni_per_capita_atlas_method_current_us'
prev = 'prevalence_of_hiv_total__of_population_ages_1549'
contr = 'condom_use_at_last_highrisk_sex_1524_years_old_women_percentage'
pvty = 'population_below_national_poverty_line_total_percentage'
abr = 'adolescent_birth_rate_per_1000_women'

## Linear model 1

Use a number of features which may be important to predict HIV incidence rate worldwide.

In [8]:
features1 = [
            'net_oda_mill', 
            pop,
            gni,
            prev,
            contr,
            pvty,
            abr,
            'year', 
            'mdgregions',
            'islldc',
            'isldc2014',
            'isdeveloped'
           ]

In [9]:
model_lr_data(hiv, features1, undata)

(15, 22)

****PREDICTING
hiv_incidence_rate_1549_years_old_percentage_midpoint
                                              OLS Regression Results                                             
Dep. Variable:     hiv_incidence_rate_1549_years_old_percentage_midpoint   R-squared:                       1.000
Model:                                                               OLS   Adj. R-squared:                  1.000
Method:                                                    Least Squares   F-statistic:                     7189.
Date:                                                   Sun, 20 Sep 2015   Prob (F-statistic):           0.000139
Time:                                                           18:04:55   Log-Likelihood:                 78.601
No. Observations:                                                     15   AIC:                            -131.2
Df Residuals:                                                          2   BIC:                            -122.0
Df Model:

/usr/local/lib/python2.7/site-packages/scipy/stats/stats.py:1233: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  int(n))


> The first model has a high R-squared, but is only using 15 observations, since many of the selected features include rows with missing values.

## Linear model 2

Limit the features so that more observations will be included. Predict HIV incidence rate worldwide.

In [10]:
features2 = [
            'net_oda_mill', 
            gni,
            prev,
            abr,
            'year', 
            'mdgregions',
            'isdeveloped'
           ]

In [11]:
model_lr_data(hiv, features2, undata)

(680, 17)

****PREDICTING
hiv_incidence_rate_1549_years_old_percentage_midpoint
                                              OLS Regression Results                                             
Dep. Variable:     hiv_incidence_rate_1549_years_old_percentage_midpoint   R-squared:                       0.817
Model:                                                               OLS   Adj. R-squared:                  0.814
Method:                                                    Least Squares   F-statistic:                     229.1
Date:                                                   Sun, 20 Sep 2015   Prob (F-statistic):          1.45e-235
Time:                                                           18:04:58   Log-Likelihood:                -87.688
No. Observations:                                                    680   AIC:                             203.4
Df Residuals:                                                        666   BIC:                             266.7
Df Model

> Contraceptive data is too sparse to make accurate predictions. Prevelance of HIV appears significant, GNI appears to be significant, adolescent birth rate is close to significant. ODA is not significant.

##Linear model 3

Sub-Saharan Africa is the area of greatest concern. Create a subset of sub-Saharan Africa data for building models. Use limited features to predict HIV incidence rate in sub-Saharan Africa.

In [12]:
ssafrica = undata[undata['mdgregions'] == 'Sub-Saharan Africa']

ssafrica.shape

(1214, 86)

In [13]:
features3 = [
            'net_oda_mill', 
            gni,
            prev,
            'year'
           ]

In [14]:
model_lr_data(hiv, features3, ssafrica)

(923, 5)

****PREDICTING
hiv_incidence_rate_1549_years_old_percentage_midpoint
                                              OLS Regression Results                                             
Dep. Variable:     hiv_incidence_rate_1549_years_old_percentage_midpoint   R-squared:                       0.760
Model:                                                               OLS   Adj. R-squared:                  0.759
Method:                                                    Least Squares   F-statistic:                     726.7
Date:                                                   Sun, 20 Sep 2015   Prob (F-statistic):          1.19e-282
Time:                                                           18:05:01   Log-Likelihood:                -710.84
No. Observations:                                                    923   AIC:                             1432.
Df Residuals:                                                        918   BIC:                             1456.
Df Model:

> ODA received and GNI do not appear to be significant. Preveleance of HIV and year are the most significant.

### Create new column - HIV percent change

Create new column to use as dependent variable, `hiv_pctchange`, which indicates the percent change in the HIV incidence rate from the previous year.

In [15]:
keys = ['countryname', 'iso3code', 'year', 'mdgregions', 'islldc', 'isldc2014', 'ismdgcountry']

In [16]:
df1 = undata[keys + [hiv]]
df1 = df1.set_index(keys)

In [17]:
df1 = df1.sort()
df2 = df1.pct_change()
df2 = df2.reset_index()
df2.columns = keys + ['hiv_pctchange']
df2.tail()

,countryname,iso3code,year,mdgregions,islldc,isldc2014,ismdgcountry,hiv_pctchange
5251,Zimbabwe,ZWE,2009,Sub-Saharan Africa,0,0,1,-0.028369
5252,Zimbabwe,ZWE,2010,Sub-Saharan Africa,0,0,1,-0.051095
5253,Zimbabwe,ZWE,2011,Sub-Saharan Africa,0,0,1,-0.069231
5254,Zimbabwe,ZWE,2012,Sub-Saharan Africa,0,0,1,-0.090909
5255,Zimbabwe,ZWE,2013,Sub-Saharan Africa,0,0,1,-0.118182


In [18]:
undata.shape

(5256, 86)

In [19]:
undata2 = pd.merge(undata, df2, how='outer', on=keys)
undata2.shape

(5256, 87)

## Linear model 4

Predict on new variable, percent change of HIV incidence rate, in sub-Saharan Africa.

In [20]:
ssafrica2 = undata2[undata2['mdgregions'] == 'Sub-Saharan Africa']

In [21]:
features4 = [
            'net_oda_mill', 
            gni,
            prev,
            abr,
            contr,
            'year', 
            'isdeveloped'
           ]

In [22]:
model_lr_data('hiv_pctchange', features4, undata2)

(30, 8)

****PREDICTING
hiv_pctchange
                            OLS Regression Results                            
Dep. Variable:          hiv_pctchange   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                 -0.096
Method:                 Least Squares   F-statistic:                    0.6386
Date:                Sun, 20 Sep 2015   Prob (F-statistic):              0.719
Time:                        18:05:12   Log-Likelihood:                -12.131
No. Observations:                  30   AIC:                             40.26
Df Residuals:                      22   BIC:                             51.47
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------

> R-squared is very low at 0.17, no features appear to be significant, and the model is trained on very few observations.

# Classification models

Convert to a classification problem and investigate feature importance.

Is HIV incidence rate decreasing from previous year?

In [23]:
undata3 = copy.deepcopy(undata2[np.isfinite(undata2['hiv_pctchange'])])
undata3.shape

(1977, 87)

Create new variable `isdecrease`.

In [24]:
isdecrease = []

for change in undata3['hiv_pctchange']:
    if change >= 0:
        isdecrease.append(0)
    if change < 0:
        isdecrease.append(1)
        
len(isdecrease)

1977

In [25]:
undata3['isdecrease'] = isdecrease

### Classification

Create functions to run multiple classification models at once and output metrics to assess accuracy.

In [26]:
rs = 6

models = {'logistic': LogisticRegression(),
          'naive bayes': GaussianNB(),
          'SVM': SVC(random_state=rs, probability=True),
          'gradient boosting': GradientBoostingClassifier(n_estimators=250, random_state=rs),
          'decision tree': DecisionTreeClassifier(random_state=rs),
          'random forest': RandomForestClassifier(n_estimators=250, random_state=rs),
          'extra trees': ExtraTreesClassifier(n_estimators=250, random_state=rs)
         }

In [27]:
def process_data(dep, ind, data):
    formula = dep + '~' + '+'.join(ind)
    y, X = dmatrices(formula, data=data, return_type='dataframe')
    X = X.iloc[:,1:]
    y = y.iloc[:,0]
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = 0.25, random_state=rs)
    return X_tr, X_ts, y_tr, y_ts

In [28]:
def get_scores(model_dict):
    for mname, m in model_dict.iteritems():
        print "*** %s" % mname
        m.fit(X_tr, y_tr)
        preds = m.predict(X_ts)
        proba = m.predict_proba(X_ts)
        print 'accuracy: %f' % accuracy_score(y_ts, preds)
        print 'precision: %f' % precision_score(y_ts, preds)
        print 'recall: %f' % recall_score(y_ts, preds)
        print 'f1 score: %f' % f1_score(y_ts, preds)
        print '\n'
        all_preds[mname] = preds
        all_proba[mname] = proba

In [29]:
def get_crossval_scores(X, y, model_dict):
    print 'CROSS VALIDATION SCORES'
    for mname, m in model_dict.iteritems():
        print '\n*** %s' % mname
        acc = np.mean(cross_val_score(m, X, y, scoring='accuracy'))
        pre = np.mean(cross_val_score(m, X, y, scoring='precision'))
        rec = np.mean(cross_val_score(m, X, y, scoring='recall'))
        f1 = np.mean(cross_val_score(m, X, y, scoring='f1'))
        print 'cv score: %f' % np.mean(cross_val_score(m, X, y))
        print 'accuracy: %f' % acc
        print 'precision: %f' % pre
        print 'recall: %f' % rec
        print 'f1 score: %f' % f1

In [30]:
ssafrica3 = undata3[undata3['mdgregions'] == 'Sub-Saharan Africa']

In [31]:
features5 = [
            'net_oda_mill', 
            gni,
            prev,
            'isdeveloped'
           ]

In [32]:
X_tr, X_ts, y_tr, y_ts = process_data('isdecrease', features5, ssafrica3)

X_tr.shape, X_ts.shape

((692, 4), (231, 4))

In [33]:
all_preds = {}
all_proba = {}

get_scores(models)

*** SVM
accuracy: 0.554113
precision: 0.550218
recall: 1.000000
f1 score: 0.709859


*** extra trees
accuracy: 0.740260
precision: 0.750000
recall: 0.785714
f1 score: 0.767442


*** decision tree
accuracy: 0.718615
precision: 0.747967
recall: 0.730159
f1 score: 0.738956


*** naive bayes
accuracy: 0.606061
precision: 0.733333
recall: 0.436508
f1 score: 0.547264


*** gradient boosting
accuracy: 0.675325
precision: 0.691729
recall: 0.730159
f1 score: 0.710425


*** logistic
accuracy: 0.675325
precision: 0.694656
recall: 0.722222
f1 score: 0.708171


*** random forest
accuracy: 0.744589
precision: 0.744526
recall: 0.809524
f1 score: 0.775665




In [34]:
get_crossval_scores(X_tr, y_tr, models)

CROSS VALIDATION SCORES

*** SVM
cv score: 0.580934
accuracy: 0.580934
precision: 0.581396
recall: 0.995025
f1 score: 0.733940

*** extra trees
cv score: 0.722592
accuracy: 0.722592
precision: 0.747222
recall: 0.788557
f1 score: 0.767094

*** decision tree
cv score: 0.645982
accuracy: 0.645982
precision: 0.691577
recall: 0.703980
f1 score: 0.697372

*** naive bayes
cv score: 0.560769
accuracy: 0.560769
precision: 0.715686
recall: 0.405473
f1 score: 0.517401

*** gradient boosting
cv score: 0.682101
accuracy: 0.682101
precision: 0.712854
recall: 0.758706
f1 score: 0.735011

*** logistic
cv score: 0.682063
accuracy: 0.682063
precision: 0.732912
recall: 0.713930
f1 score: 0.723011

*** random forest
cv score: 0.726915
accuracy: 0.726915
precision: 0.750441
recall: 0.793532
f1 score: 0.771371


### Investigate feature importance

Feature importance for best models, Extra Trees, Random Forest, and Gradient Boosting.

In [35]:
trees = ExtraTreesClassifier(random_state=rs)

trees.fit(X_tr, y_tr)
importances = trees.feature_importances_

indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(len(features5) - 1):
    print("%d. feature: %s (%f)" % (f + 1, features5[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature: prevalence_of_hiv_total__of_population_ages_1549 (0.396860)
2. feature: net_oda_mill (0.306375)
3. feature: gni_per_capita_atlas_method_current_us (0.296765)


In [36]:
forest = RandomForestClassifier(random_state=rs)

forest.fit(X_tr, y_tr)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(len(features5) - 1):
    print("%d. feature: %s (%f)" % (f + 1, features5[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature: prevalence_of_hiv_total__of_population_ages_1549 (0.399475)
2. feature: net_oda_mill (0.338189)
3. feature: gni_per_capita_atlas_method_current_us (0.262336)


In [37]:
gradient = GradientBoostingClassifier(random_state=rs)

gradient.fit(X_tr, y_tr)
importances = gradient.feature_importances_

indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(len(features5) - 1):
    print("%d. feature: %s (%f)" % (f + 1, features5[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature: net_oda_mill (0.424601)
2. feature: prevalence_of_hiv_total__of_population_ages_1549 (0.296408)
3. feature: gni_per_capita_atlas_method_current_us (0.278991)


> The Extra Trees Classifier, Random Forest, and Grandient Boosting Classifier perform reasonably well. The Gradient Boosting model is the only model with ODA as the most important feature, which warrants further investigation.